## Install dependencies

In [1]:
!pip install tf-nightly-gpu-2.0-preview==2.0.0-dev20190504

import tensorflow as tf
print(tf.__version__)


     |████████████████████████████████| 346.5MB 59kB/s 
     |████████████████████████████████| 430kB 44.5MB/s 
     |████████████████████████████████| 61kB 27.2MB/s 
     |████████████████████████████████| 3.1MB 40.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/89/67/41/63cbf0f6ac0a6156588b9587be4db5565f8c6d8ccef98202fc
Successfully built wrapt
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
  Found existing installation: wrapt 1.10.11
    Uninstalling wrapt-1.10.11:
      Successfully uninstalled wrapt-1.10.11
2.0.0-dev20190504


## Integrate Google Drive

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


## Update Git Repository 

In [0]:
%cd /gdrive/Team\ Drives/umayux/Research/NLP/humor_detection/
!git pull



/gdrive/Team Drives/umayux/Research/NLP/humor_detection
From https://github.com/gaarangoa/transformer_classifier
   a718d96..691665f  master     -> origin/master
Updating a718d96..691665f


## Train the model

In [7]:
%cd /gdrive/Team\ Drives/umayux/Research/NLP/humor_detection/
!python train.py --dataset_file ./data/dataset.tsv --checkpoint_path ./data/ --BATCH_SIZE 1000 --retrain

/gdrive/Team Drives/umayux/Research/NLP/humor_detection
[2019-05-04 14:40:39,560] INFO - start
[2019-05-04 14:40:41,906] INFO - saving parameters to ./data/
[2019-05-04 14:40:41,910] INFO - loading dataset
2019-05-04 14:40:42.345107: I tensorflow/stream_executor/platform/default/dso_loader.cc:42] Successfully opened dynamic library libcuda.so.1
2019-05-04 14:40:42.414756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-04 14:40:42.415174: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Found device 0 with properties: 
name: Tesla T4 major: 7 minor: 5 memoryClockRate(GHz): 1.59
pciBusID: 0000:00:04.0
2019-05-04 14:40:42.415266: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1005] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-05-04

## Test the model

In [0]:
%cd /gdrive/Team\ Drives/umayux/Research/NLP/chatbot/transformer/

import tensorflow_datasets as tfds
import tensorflow as tf
import utensor.dataset as dt
from utensor.optimizer import CustomSchedule, loss_function
from utensor.model import Transformer
import time
from utensor.masking import create_masks
import pickle
import matplotlib.pyplot as plt



checkpoint_path="./data/banco/"
d_model = 128
MAX_LENGTH=60
BUFFER_SIZE=20000
BATCH_SIZE=64
num_heads=8
num_layers=4
d_model=128
dff=512
dropout_rate=0.1



def restore():
    
    # loading tokenizers for future predictions
    tokenizer_source = pickle.load(open(checkpoint_path + './tokenizer_source.pickle', 'rb'))
    tokenizer_target = pickle.load(open(checkpoint_path + './tokenizer_target.pickle', 'rb'))

    input_vocab_size = tokenizer_source.vocab_size + 2
    target_vocab_size = tokenizer_target.vocab_size + 2

    learning_rate = CustomSchedule(d_model)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
    )

    transformer = Transformer(
        num_layers,
        d_model,
        num_heads,
        dff,
        input_vocab_size,
        target_vocab_size,
        dropout_rate,
    )


    ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

    # if a checkpoint exists, restore the latest checkpoint.
    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print("Latest checkpoint restored!!")
    else:
        print("Initializing from scratch.")
        
    return transformer, tokenizer_source, tokenizer_target


           
    
def evaluate(inp_sentence):
    start_token = [tokenizer_source.vocab_size]
    end_token = [tokenizer_source.vocab_size + 1]

    # inp sentence is portuguese, hence adding the start and end token
    inp_sentence = start_token + tokenizer_source.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    # as the target is english, the first word to the transformer should be the
    # english start token.
    decoder_input = [tokenizer_target.vocab_size]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(MAX_LENGTH):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
            encoder_input, output)

        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer(encoder_input, 
                                                     output,
                                                     False,
                                                     enc_padding_mask,
                                                     combined_mask,
                                                     dec_padding_mask)

        # select the last word from the seq_len dimension
        predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # return the result if the predicted_id is equal to the end token
        if tf.equal(predicted_id, tokenizer_target.vocab_size+1):
            return tf.squeeze(output, axis=0), attention_weights

        # concatentate the predicted_id to the output which is given to the decoder
        # as its input.
        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights



def translate(sentence):
    result, attention_weights = evaluate(sentence)

    predicted_sentence = tokenizer_target.decode([i for i in result 
                                            if i < tokenizer_target.vocab_size])  

    print('Pregunta: {}'.format(sentence))
    print('Respuesta UmyBot: {}'.format(predicted_sentence))



transformer, tokenizer_source, tokenizer_target = restore()


/gdrive/Team Drives/umayux/Research/NLP/chatbot/transformer
Latest checkpoint restored!!


In [0]:
translate('banco_falabella dicen que no están operativo')

Pregunta: banco_falabella dicen que no están operativo
Respuesta UmyBot: hola @fefith, hemos realizado pruebas de transferencia y no presentamos inconvenientes.


In [0]:
translate('@bancolombia #bancoeterno se cagaron literalmente el servicio. ahora son menos filas pero más demoras con el servicio.')

Pregunta: @bancolombia #bancoeterno se cagaron literalmente el servicio. ahora son menos filas pero más demoras con el servicio.
Respuesta UmyBot: ¡hola! cuéntanos por favor si haces referencia a un poco más de tu comentario, para poder ayudarte. saludos. ana


## Predict examples

In [0]:
%cd /gdrive/Team\ Drives/umayux/Research/NLP/chatbot/transformer/

import tensorflow_datasets as tfds
import tensorflow as tf
import utensor.dataset as dt
from utensor.optimizer import CustomSchedule, loss_function
from utensor.model import Transformer
import time
from utensor.masking import create_masks
import pickle
import matplotlib.pyplot as plt

checkpoint_path="./data/banco/"
d_model = 128
MAX_LENGTH=40
BUFFER_SIZE=20000
BATCH_SIZE=64
num_heads=8
num_layers=4
dff=512
dropout_rate=0.1

def restore():

    # loading tokenizers for future predictions
    tokenizer_source = pickle.load(open(checkpoint_path+'/tokenizer_source.pickle', 'rb'))
    tokenizer_target = pickle.load(open(checkpoint_path+'/tokenizer_target.pickle', 'rb'))

    input_vocab_size = tokenizer_source.vocab_size + 2
    target_vocab_size = tokenizer_target.vocab_size + 2

    learning_rate = CustomSchedule(d_model)
    optimizer = tf.keras.optimizers.Adam(
        learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9
    )

    transformer = Transformer(
        num_layers,
        d_model,
        num_heads,
        dff,
        input_vocab_size,
        target_vocab_size,
        dropout_rate,
    )

    ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)
    ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

    # if a checkpoint exists, restore the latest checkpoint.
    if ckpt_manager.latest_checkpoint:
        ckpt.restore(ckpt_manager.latest_checkpoint)
        print("Latest checkpoint restored!!")
    else:
        print("Initializing from scratch.")

    return transformer, tokenizer_source, tokenizer_target
        

        
def evaluate(inp_sentence):
    start_token = [tokenizer_source.vocab_size]
    end_token = [tokenizer_source.vocab_size + 1]
    
    
    # inp sentence is portuguese, hence adding the start and end token
    inp_sentence = start_token + tokenizer_source.encode(inp_sentence) + end_token
    encoder_input = tf.expand_dims(inp_sentence, 0)

    # as the target is english, the first word to the transformer should be the
    # english start token.
    decoder_input = [tokenizer_target.vocab_size]
    output = tf.expand_dims(decoder_input, 0)

    for i in range(40):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)

    # select the last word from the seq_len dimension
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # return the result if the predicted_id is equal to the end token
    if tf.equal(predicted_id, tokenizer_target.vocab_size+1):
        return tf.squeeze(output, axis=0), attention_weights

    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights




def translate(sentence, plot=''):
    
    result, attention_weights = evaluate(sentence)

    predicted_sentence = tokenizer_target.decode([i for i in result 
                                            if i < tokenizer_target.vocab_size])  

    print('Pregunta: {}'.format(sentence))
    print('Respuesta UmyBot: {}'.format(predicted_sentence))

    #   if plot:
    #     plot_attention_weights(attention_weights, sentence, result, plot)

    
    
transformer, tokenizer_source, tokenizer_target = restore()


/gdrive/Team Drives/umayux/Research/NLP/chatbot/transformer
Latest checkpoint restored!!


In [0]:
sentence = '@bancolombia #bancoeterno se cagaron literalmente el servicio. ahora son menos filas pero más demoras con el servicio.'
translate(sentence, plot='')

Pregunta: @bancolombia #bancoeterno se cagaron literalmente el servicio. ahora son menos filas pero más demoras con el servicio.
Respuesta UmyBot: ¡


In [0]:
import pandas as pd
data = pd.read_csv('./data/banco/bancobot.tsv.test', sep='\t', names=['source', 'target'])



In [0]:

for ix,i in data.iterrows():
    translate(
        i['source']
    )
    print("Respuesta Humano: {}".format(i['target']))
    print('\n\n')
    


In [0]:
def plot_attention_weights(attention, sentence, result, layer, tokenizer_source, tokenizer_target):
  fig = plt.figure(figsize=(30, 38))
  
  sentence = tokenizer_source.encode(sentence)
  
  attention = tf.squeeze(attention[layer], axis=0)
  
  for head in range(attention.shape[0]):
    ax = fig.add_subplot(8, 1, head+1)
    
    # plot the attention weights
    ax.matshow(attention[head][:-1, :], cmap='viridis')

    fontdict = {'fontsize': 10}
    
    ax.set_xticks(range(len(sentence)+2))
    ax.set_yticks(range(len(result)))
    
    ax.set_ylim(len(result)-1.5, -0.5)
        
    ax.set_xticklabels(
        ['<start>']+[tokenizer_source.decode([i]) for i in sentence]+['<end>'], 
        fontdict=fontdict, rotation=90)
    
    ax.set_yticklabels([tokenizer_target.decode([i]) for i in result 
                        if i < tokenizer_target.vocab_size], 
                       fontdict=fontdict)
    
    ax.set_xlabel('Head {}'.format(head+1))
  
  plt.tight_layout()
  plt.show()